In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import data as DATADATA
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM
from keras.models import clone_model
from sklearn.preprocessing import MinMaxScaler
import os

os.makedirs('models-KaggleSMA', exist_ok=True)

tickers = ['SOLUSDT', 'BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT']
scalers = {}  # Save scalers per ticker for later testing

# Step 1: Build combined training dataset
combined_x_train = []
combined_y_train = []

for ticker in tickers:
    df = DATADATA.load_asset(ticker, sampling='1d')
    df = DATADATA.subset(df, start=pd.Timestamp('2022-01-01'), end=pd.Timestamp('2024-03-01'))
    df['Returns'] = df['Close'].pct_change()
    df.dropna(inplace=True)

    data = df.filter(['Returns'])
    dataset = data.values

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)
    scalers[ticker] = scaler

    for i in range(60, len(scaled_data)):
        combined_x_train.append(scaled_data[i - 60:i, 0])
        combined_y_train.append(scaled_data[i, 0])

# Convert combined dataset to numpy arrays
combined_x_train = np.array(combined_x_train)
combined_y_train = np.array(combined_y_train)
combined_x_train = np.reshape(combined_x_train, (combined_x_train.shape[0], combined_x_train.shape[1], 1))

# Step 2: Train base model
base_model = Sequential()
base_model.add(LSTM(128, return_sequences=True, input_shape=(60, 1)))
base_model.add(LSTM(64, return_sequences=False))
base_model.add(Dense(25))
base_model.add(Dense(1))
base_model.compile(optimizer='adam', loss='mean_squared_error')

base_model.fit(combined_x_train, combined_y_train, batch_size=8, epochs=16)

# Save base model (optional)
base_model.save('models-KaggleSMA/base_model.keras')

Epoch 1/16


c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


457/457 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 0.0132
Epoch 2/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0074
Epoch 3/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0066
Epoch 4/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0081
Epoch 5/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0071
Epoch 6/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0066
Epoch 7/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0068
Epoch 8/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0068
Epoch 9/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0071
Epoch 10/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0076
Epoch 11/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0067
Epoch 12/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0069
Epoch 13/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0062
Epoch 14/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0077
Epoch 15/16
457/457 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step 

In [2]:

# Step 3: Fine-tune separate model per ticker
for ticker in tickers:
    df = DATADATA.load_asset(ticker, sampling='1d')
    df = DATADATA.subset(df, start=pd.Timestamp('2022-01-01'), end=pd.Timestamp('2024-03-01'))
    df['Returns'] = df['Close'].pct_change()
    df.dropna(inplace=True)

    data = df.filter(['Returns'])
    dataset = data.values

    scaler = scalers[ticker]
    scaled_data = scaler.transform(dataset)

    x_train = []
    y_train = []
    for i in range(60, len(scaled_data)):
        x_train.append(scaled_data[i - 60:i, 0])
        y_train.append(scaled_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    # Clone base model architecture and load weights
    fine_tuned_model = clone_model(base_model)
    fine_tuned_model.set_weights(base_model.get_weights())

    # Freeze base layers
    for layer in fine_tuned_model.layers:
        layer.trainable = False

    # Add a new trainable output layer
    model = Sequential()
    for layer in fine_tuned_model.layers[:-1]:
        model.add(layer)
    model.add(Dense(1))  # New output layer

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, batch_size=8, epochs=8)

    # Save fine-tuned model
    model.save(f'models-KaggleSMA/lstm_transferred_model_{ticker}.keras')

    predictions = model.predict(x_train)
    predictions = scaler.inverse_transform(predictions)
    
    valid = data[60:]
    valid['Predictions'] = predictions
    display(valid.head(10))


Epoch 1/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.1929
Epoch 2/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0599
Epoch 3/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0162
Epoch 4/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0065
Epoch 5/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0060
Epoch 6/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0071
Epoch 7/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0054
Epoch 8/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0060
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


C:\Users\ic2594\AppData\Local\Temp\ipykernel_16676\1810716248.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


,Returns,Predictions
Open time,,
2022-03-03,-0.050258,0.001995
2022-03-04,-0.076344,0.001341
2022-03-05,0.014832,-0.000270
2022-03-06,-0.056454,-0.000358
2022-03-07,-0.034409,-0.001060
2022-03-08,0.008082,-0.001794
2022-03-09,0.069728,-0.001699
2022-03-10,-0.058937,-0.000188
2022-03-11,-0.026548,-0.000611


Epoch 1/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.1908
Epoch 2/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0662
Epoch 3/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0184
Epoch 4/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0102
Epoch 5/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0083
Epoch 6/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0094
Epoch 7/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0087
Epoch 8/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0096
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


C:\Users\ic2594\AppData\Local\Temp\ipykernel_16676\1810716248.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


...

C:\Users\ic2594\AppData\Local\Temp\ipykernel_16676\1810716248.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


,Returns,Predictions
Open time,,
2022-03-03,-0.022375,0.000968
2022-03-04,-0.051630,0.000754
2022-03-05,0.057949,0.000370
2022-03-06,-0.038329,0.000865
2022-03-07,-0.007447,0.000349
2022-03-08,0.001389,0.000281
2022-03-09,0.062578,0.000372
2022-03-10,-0.038783,0.000884
2022-03-11,0.090749,0.000455


Epoch 1/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0624
Epoch 2/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0153
Epoch 3/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0069
Epoch 4/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0040
Epoch 5/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0051
Epoch 6/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0057
Epoch 7/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0055
Epoch 8/8
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0051
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


C:\Users\ic2594\AppData\Local\Temp\ipykernel_16676\1810716248.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


,Returns,Predictions
Open time,,
2022-03-03,-0.024812,-0.003167
2022-03-04,-0.054742,-0.003771
2022-03-05,0.019576,-0.005031
2022-03-06,-0.034400,-0.005585
2022-03-07,-0.033140,-0.005608
2022-03-08,0.002571,-0.005926
2022-03-09,0.038462,-0.006014
2022-03-10,-0.037860,-0.005209
2022-03-11,-0.011976,-0.004804
